In [1]:
import ee
ee.Initialize()
import geemap
import requests
import json
import geopandas as gpd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
Boston=ee.Geometry.BBox(-71.19083563,  42.22788727, -70.92346937,  42.39693964)

In [3]:
Map=geemap.Map()
Map.setCenter(-71.0589, 42.3601,zoom=15)
Map.add_basemap("HYBRID")

# Sentinel 2 Level 1C

In [4]:
S2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(Boston) \
    .filterDate('2018-05-01', '2018-06-30') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .min() \
    .select(['B2', 'B3', 'B4', 'B8', 'B9','B11','B12']) \
    .divide(10000)

In [5]:
S2.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B8',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B9',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 6.553500175476074},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B11',
   'data_type': {

In [7]:
blue=S2.select('B2')
nir = S2.select('B8')
red = S2.select('B4')

In [8]:
# EVI = 2.5 * (B08 - B04) / ((B08 + 6.0 * B04 - 7.5 * B02) + 1.0);
numerator = nir.subtract(red).multiply(ee.Number(2.5))
denominator = nir.add(red.multiply(ee.Number(6))).subtract(blue.multiply(ee.Number(7.5))).add(ee.Number(1))
evi=numerator.divide(denominator).rename('EVI')

In [9]:
evi.getInfo()

{'type': 'Image',
 'bands': [{'id': 'EVI',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [ ]:
Map.addLayer(evi,{'min': 0, 'max': 2, 'palette': ['#006a4e', '#b8d5cd','#bbbbbb']},'evi')
Map